In [1]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)

from src.settings import load_env, DATA_DIR

load_env()

True

In [2]:
with open(DATA_DIR / "sample_legis.txt") as f:
    legis_text = f.read()

legis_text[:100] + " ..."

'Sygnatura akt II AKa 335/15\n\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 20 stycznia 2016 r.\n\nSąd ...'

In [3]:
INSTRUCTION = '''\
You are a question-answer generator. Your goal is to generate question-answer pairs given the Context.
Do not tranlate the Context, generate questions and answers in original language.

Example output: {{"question": "generated question", "answer": "gathered answer"}}

Context: {context}

Step 1: Identify spans that are likely to be answers to questions, identify as many as possible.
Step 2: For each identified span, generate a question.
Step 3: Respond to the question in only a few tokens concisely.
Step 4: Output in JSON format following [...]

Ensure that you distinctly label and delineate Steps 1, 2, 3, and 4.

{format_instructions}
'''

In [4]:
# Define your desired data structure.
class SyntheticLegisQARecord(BaseModel):
    question: str = Field(description="Question parsed from last step of the prompt")
    answer: str = Field(description="Answer parsed from last step of the prompt")

In [5]:
output_parser = JsonOutputParser(pydantic_object=SyntheticLegisQARecord)

In [6]:
prompt = ChatPromptTemplate.from_template(
    template=INSTRUCTION,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [7]:
model = ChatOpenAI(model="gpt-4")

chain = prompt | model | output_parser

chain.invoke({"context": legis_text})

[{'question': 'Jaki sąd wydał ten wyrok?',
  'answer': 'Sąd Apelacyjny we Wrocławiu II Wydział Karny'},
 {'question': 'Kiedy wydano ten wyrok?', 'answer': '20 stycznia 2016 r.'},
 {'question': 'Kto był przewodniczącym podczas wydania wyroku?',
  'answer': 'SSA Stanisław Rączkowski'},
 {'question': 'Kto był sędzią podczas wydania wyroku?',
  'answer': 'SSA Edward Stelmasik'},
 {'question': 'Kto był protokołantem podczas wydania wyroku?',
  'answer': 'Beata Sienica'},
 {'question': 'Kto był prokuratorem podczas wydania wyroku?',
  'answer': 'Leszka Pruskiego'},
 {'question': 'O jaką sprawę chodziło w tym wyroku?',
  'answer': 'sprawa G. K.'},
 {'question': 'Za jaki artykuł oskarżono G. K.?', 'answer': 'art. 280 § 2 kk'},
 {'question': 'Od jakiego wyroku wniesiono apelację?',
  'answer': 'wyrok Sądu Okręgowego w Jeleniej Górze z dnia 20 października 2015 r. sygn. akt III K 43/15'},
 {'question': 'Jaką karę wymierzono oskarżonemu G. K.?',
  'answer': '3 ( trzech) lat pozbawienia wolności'}